In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx=QQ['xx'].0
(I-sqrt(3)).minpoly(xx)

xx^4 - 4*xx^2 + 16

In [3]:
Q.<i> = NumberField(xx^2 + 1)

In [4]:
#i = ((rad2_i)^3+rad2_i)/(-6)

In [5]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [6]:
mon = ((x+y+z+t)^3).monomials()
L_set_base = ['E1', 'G4', 'E2', 'G3', 'E3']

In [7]:
%time adm_SE1 = SE1.find_admissible_projectivities()
len(adm_SE1)

CPU times: user 8.24 s, sys: 301 ms, total: 8.54 s
Wall time: 21 s


576

In [8]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)
len(simm_SE1)

CPU times: user 4.55 s, sys: 223 ms, total: 4.77 s
Wall time: 24.1 s


2

In [9]:
lines_perms = []
for simm in simm_SE1:
    lines_perms.append(Permutation(SE1.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(2, 'C2')

In [10]:
adm_perms = [from_labels_to_perm(label) for label in SE1.find_admissible_permutations()]
M2_perm = lines_perms_group.gens_small()[0]
adm_perms_group = PermutationGroup(adm_perms)

In [11]:
M2_perm

(2,19)(3,22)(4,7)(5,25)(6,26)(8,13)(9,14)(11,16)(12,17)(18,27)(20,24)(21,23)

In [12]:
# find permutation of order 4 whose square is M2_perm
possible_perms = []
for perm in adm_perms_group:
    if perm^2 == M2_perm and perm.order() == 4:
        possible_perms.append(perm)
len(possible_perms)

12

In [13]:
possible_L_sets = [get_permuted_L_set(perm) for perm in possible_perms]
possible_L_sets

[('E1', 'G4', 'E5', 'F15', 'E6'),
 ('E1', 'G4', 'F45', 'G5', 'F46'),
 ('E1', 'G4', 'E6', 'F16', 'E5'),
 ('E1', 'G4', 'F46', 'G6', 'F45'),
 ('E1', 'G4', 'E3', 'G6', 'F24'),
 ('E1', 'G4', 'F34', 'F16', 'E2'),
 ('E1', 'G4', 'E5', 'G2', 'F46'),
 ('E1', 'G4', 'F45', 'F12', 'E6'),
 ('E1', 'G4', 'E3', 'G5', 'F24'),
 ('E1', 'G4', 'F34', 'F15', 'E2'),
 ('E1', 'G4', 'E6', 'G2', 'F45'),
 ('E1', 'G4', 'F46', 'F12', 'E5')]

In [34]:
proj = find_projectivity(SE1.get_L_set_in_plucker(L_set_base), SE1.get_L_set_in_plucker(('E1', 'G4', 'F45', 'G5', 'F46')))
nc = SE1.subs(change_coord(proj))
conds = list(set(matrix([[SE1.coefficient(mn) for mn in mon], [nc.coefficient(mn) for mn in mon]]).minors(2)))
red_conds = [remove_sing_factors(cond, SE1.sing_locus) for cond in conds if cond !=0]
gb = P.ideal(red_conds).groebner_basis()

In [36]:
gbf = gb+[f]
prim_dec1 = P.ideal(gbf).radical().primary_decomposition()

[Ideal (f, 25*d + (2*i - 11)*e, 5*c + (4*i + 3)*e) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 1,
 Ideal (f, 25*d + (-2*i - 11)*e, 5*c + (-4*i + 3)*e) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 1,
 Ideal (f, d - e, c + e) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 1,
 Ideal (f, c) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in i with defining polynomial xx^2 + 1]

In [41]:
%time prim_dec2 = P.ideal(gb).saturation(P.ideal(f))[0].radical().primary_decomposition()

CPU times: user 28.5 s, sys: 11.2 ms, total: 28.5 s
Wall time: 28.4 s


In [44]:
ideals = [ideal for ideal in prim_dec1+prim_dec2 if SE1.is_ideal_valid(ideal)]

In [82]:
gens = ideals[-1].gens()
gens

[8*c*e + (3*i - 1)*d*e + (5*i + 5)*e^2 - 4*c*f + (-3*i + 1)*d*f + (-8*i)*e*f + (3*i - 1)*f^2, 40*c*d + (7*i - 1)*d*e + (9*i + 13)*e^2 + (8*i - 4)*c*f + (9*i - 7)*d*f + (8*i - 24)*e*f + (-9*i + 7)*f^2, 8*c^2 + (-i + 3)*d*e + (-7*i + 1)*e^2 + 4*c*f + (i - 3)*d*f + (8*i - 8)*e*f + (-i + 3)*f^2, 25*d^2*e + (-48*i + 14)*d*e^2 + (24*i - 7)*e^3 - 25*d^2*f + (18*i - 74)*d*e*f + (-34*i - 13)*e^2*f + (-2*i + 36)*d*f^2 + (8*i + 31)*e*f^2 + (2*i - 11)*f^3]

In [72]:
-(gens[0] -d*gens[0].coefficient(d))/gens[0].coefficient(d)

(-8*c*e + (-5*i - 5)*e^2 + 4*c*f + (8*i)*e*f + (-3*i + 1)*f^2)/((3*i - 1)*e + (-3*i + 1)*f)

In [74]:
gens_d = [el.subs({d: -(gens[0] -d*gens[0].coefficient(d))/gens[0].coefficient(d)}) for el in gens]
gens_d

[0,
 ((1920*i + 2560)*c^2*e + (3200*i)*c*e^2 + (-960*i - 1280)*c^2*f + (-2240*i + 1280)*c*e*f + (640*i - 1280)*e^2*f + (320*i - 640)*c*f^2 + (-320*i + 640)*e*f^2)/((-18*i + 26)*e + (18*i - 26)*f),
 8*c^2 + (32/5*i + 24/5)*c*e + (-16/5*i + 8/5)*c*f + (16/5*i - 8/5)*e*f,
 ((19200*i + 505600)*c^2*e^2 + (416000*i + 288000)*c*e^3 + (-19200*i - 505600)*c^2*e*f + (-614400*i - 179200)*c*e^2*f + (198400*i - 108800)*e^3*f + (4800*i + 126400)*c^2*f^2 + (302400*i - 36800)*c*e*f^2 + (-198400*i + 108800)*e^2*f^2 + (-49600*i + 27200)*c*f^3 + (49600*i - 27200)*e*f^3)/((-1992*i - 2456)*e + (1992*i + 2456)*f)]

In [83]:
[el.subs({e: -(P(gens_d[2]) -e*P(gens_d[2]).coefficient(e))/P(gens_d[2]).coefficient(e)}) for el in gens_d]

[0, 0, 0, 0]